In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQBN8DPW2rdiRrY34eEM53HAzakNGSRrw4ogI-j8HyCUrbqTB_z4CeIn2IvjLF-w_6sOe5pIlypJGAA/pub?output=csv')
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB
None


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [11]:
smoker = df.loc[df['smoker'] == 'yes'].copy()
non_smoker = df.loc[df['smoker'] == 'no'].copy()

In [12]:
smoker_charge=smoker['charges']
non_smoker_charge=non_smoker['charges']

# Q1. Do smokers have higher insurance charges than non-smokers?


## State your Null Hypothesis and Alternative Hypothesis

- Null Hypothesis: No difference on price between smokers and non smokers.

- Alternative Hypothesis: There is difference on price between smokers and non smokers.

## Select the correct test according to the data type and number of samples

it is a two sample numeric data, so I will perform independent t test.


## Test the assumptions of your selected test.

There are three assumptions:
 - No significant outliers
 - Normality
 - Equal Variance

In [4]:
# Check for outliers for both groups
##smoker_charge
smoker_zscores= stats.zscore(smoker_charge)
smoker_outliers = abs(smoker_zscores)>3
print((np.sum(smoker_outliers)))
# remove outliers from strength group
smoker_charge = smoker_charge[(np.abs(smoker_zscores) < 3)]

0


In [13]:
##non_smoker_charge
non_smoker_zscores= stats.zscore(non_smoker_charge)
non_smoker_outliers = abs(non_smoker_zscores)>3
print((np.sum(non_smoker_outliers)))
# remove outliers from strength group
non_smoker_charge = non_smoker_charge[(np.abs(non_smoker_zscores) < 3)]

24


In [14]:
# test the strength group for normality
result_smoker_charge= stats.normaltest(smoker_charge)
result_non_smoker_charge = stats.normaltest(non_smoker_charge)

print(result_smoker_charge)
print(result_non_smoker_charge)

NormaltestResult(statistic=61.03941356533816, pvalue=5.564930630036463e-14)
NormaltestResult(statistic=163.80367047789198, pvalue=2.6945416315543976e-36)


In [17]:
# check for equal variance
# Test for equal variance
result = stats.ttest_ind(smoker_charge, non_smoker_charge)
result

Ttest_indResult(statistic=51.2078044173717, pvalue=3.68768124e-315)

result suggest we don't have equal variances, so we just need to be sure to include "equal_var = False" when we perform our t-test.

## Execute the selected test, or the alternative test (if you do not meet the assumptions)

In [15]:
# Independent t-test with equal_var set to False
result = stats.ttest_ind(smoker_charge, non_smoker_charge, equal_var = False)
result

Ttest_indResult(statistic=33.732305987092516, pvalue=2.575322662587431e-104)

## Interpret your p-value and reject or fail to reject your null hypothesis 


Our p-value < alpha (0.05), so we reject the null hypothesis and accept that there is a significant difference on price between smokers and non smokers.

## Show a supporting visualization that helps display the result

# Q2. Are men more likely to smoke than women?

## State your Null Hypothesis and Alternative Hypothesis

- Null Hypothesis: men and women are equally likely to smoke

- Alternative Hypothesis: men and women are not equally likely to smoke


# Q3. Do different regions have different charges, on average?

## State your Null Hypothesis and Alternative Hypothesis


- Null Hypothesis: different regions have the same charges in average.

- Alternative Hypothesis: different regions have different charges in average.